In [3]:
import pandas as pd
import torch
import transformers
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset
from sklearn.model_selection import train_test_split

In [4]:
# Load dataset
df = pd.read_csv("Dataset.csv")

# Ensure correct column names
df = df[['context', 'response']]
df = df.dropna()

In [5]:
def format_conversations(row):
    return f"User: {row['context']}\nBot: {row['response']}"

df["formatted"] = df.apply(format_conversations, axis=1)


In [6]:
# Split into train and validation sets
train_texts, val_texts = train_test_split(df["formatted"].tolist(), test_size=0.1, random_state=42)


In [7]:
# Load tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained("gpt2")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [8]:
# Convert to Hugging Face dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

train_dataset = Dataset.from_dict({"text": train_texts}).map(tokenize_function, batched=True)
val_dataset = Dataset.from_dict({"text": val_texts}).map(tokenize_function, batched=True)


Map:   0%|          | 0/61298 [00:00<?, ? examples/s]

Map:   0%|          | 0/6811 [00:00<?, ? examples/s]

In [9]:
# Data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


In [10]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-chatbot",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=500,
    fp16=torch.cuda.is_available(),
    save_total_limit=2,
    report_to="none"
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

# Train model
trainer.train()


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-10-5d2cea76c1db>:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,2.589000,2.588970
2,2.398100,2.508463
3,2.270100,2.487255


TrainOutput(global_step=45975, training_loss=2.457798470211392, metrics={'train_runtime': 12945.9161, 'train_samples_per_second': 14.205, 'train_steps_per_second': 3.551, 'total_flos': 4.8050036932608e+16, 'train_loss': 2.457798470211392, 'epoch': 3.0})

In [11]:
# Save final model
model.save_pretrained("./gpt2-chatbot-final")
tokenizer.save_pretrained("./gpt2-chatbot-final")


('./gpt2-chatbot-final/tokenizer_config.json',
 './gpt2-chatbot-final/special_tokens_map.json',
 './gpt2-chatbot-final/vocab.json',
 './gpt2-chatbot-final/merges.txt',
 './gpt2-chatbot-final/added_tokens.json')

In [12]:
!zip -r gpt2-chatbot-final.zip ./gpt2-chatbot-final


  adding: gpt2-chatbot-final/ (stored 0%)
  adding: gpt2-chatbot-final/merges.txt (deflated 53%)
  adding: gpt2-chatbot-final/config.json (deflated 51%)
  adding: gpt2-chatbot-final/tokenizer_config.json (deflated 56%)
  adding: gpt2-chatbot-final/vocab.json (deflated 68%)
  adding: gpt2-chatbot-final/generation_config.json (deflated 24%)
  adding: gpt2-chatbot-final/model.safetensors (deflated 7%)
  adding: gpt2-chatbot-final/special_tokens_map.json (deflated 74%)


In [13]:
from google.colab import files
files.download("gpt2-chatbot-final.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>